# Q1. Running Elastic

In [28]:
!curl localhost:9200

{
  "name" : "16165911bfe7",
  "cluster_name" : "docker-cluster",
  "cluster_uuid" : "1nlL9fbyThK179peW65kSQ",
  "version" : {
    "number" : "8.4.3",
    "build_flavor" : "default",
    "build_type" : "docker",
    "build_hash" : "42f05b9372a9a4a470db3b52817899b99a76ee73",
    "build_date" : "2022-10-04T07:17:24.662462378Z",
    "build_snapshot" : false,
    "lucene_version" : "9.3.0",
    "minimum_wire_compatibility_version" : "7.17.0",
    "minimum_index_compatibility_version" : "7.0.0"
  },
  "tagline" : "You Know, for Search"
}


### What's the `version.build_hash` value?

`42f05b9372a9a4a470db3b52817899b99a76ee73`

# Q2. Indexing the data

In [16]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [5]:
from elasticsearch import elasticsearch

In [8]:
es_client = Elasticsearch('http://localhost:9200')

In [14]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "selection": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"}
        }
    }
}

index_name = "course-questions"

es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [19]:
from tqdm.auto import tqdm

for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

/usr/local/python/3.10.13/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 948/948 [00:31<00:00, 30.04it/s]


### Which function do you use for adding your data to elastic?

`index`

# Q3. Searching

In [54]:
query = "How do I execute a command in a running docker container?"

search_query = {
    "size": 5,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text"], # use only "text" and "question" with boost of 4
                    "type": "best_fields" 
                }
            },
            "filter": {
                "term": {
                    "course": "data-engineering-zoomcamp"
                }
            }
        }
    }
}

response = es_client.search(index=index_name, body=search_query)
response['hits']['hits'][0]['_score']

75.54128

### What's the score for the top ranking result?

* 94.05
* 84.05
* `74.05` <-- closest to my result (`75.54`)
* 64.05

# Q4. Filtering


In [62]:
query = "How do I execute a command in a running docker container?"

search_query = {
    "size": 3,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text"], # use only "text" and "question" with boost of 4
                    "type": "best_fields" 
                }
            },
            "filter": {
                "term": {
                    "course": "machine-learning-zoomcamp" # machine-learning-zoomcamp
                }
            }
        }
    }
}

response = es_client.search(index=index_name, body=search_query)
response['hits']['hits'][-1]['_source']['question']

'How do I copy files from a different folder into docker container’s working directory?'

### Return 3 results. What's the 3rd question returned by the search engine?

* How do I debug a docker container?
* `How do I copy files from a different folder into docker container’s working directory?`
* How do Lambda container images work?
* How can I annotate a graph?

# Q5. Building a prompt


In [65]:
q5_docs = []

for doc in response['hits']['hits']:
    q5_docs.append(doc['_source'])

q5_docs

[{'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I debug a docker container?',
  'course': 'machine-learning-zoomcamp'},
 {'text': "You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:\nTo copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:\ndocker cp /path/to/local/file_or_directory container_id:/path/in/container\nHrithik Kumar Advani",
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I copy files from my local machine to docker container?',
 

In [84]:
context_template = """
Q: {question}
A: {text}
""".strip()

prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

In [87]:
def build_prompt(query, search_results):
    prompt_temp = prompt_template
    context = ""

    for doc in search_results:
        context = context + context_template.format(question=doc['question'], text=doc['text'])
    
    return prompt_temp.format(question=query, context=context).strip()

In [93]:
prompt = build_prompt(query, q5_docs)

In [94]:
len(prompt)

1458

What's the length of the resulting prompt? (use the `len` function)

* 962
* `1462`
* 1962
* 2462

# Q6. Tokens

In [91]:
import tiktoken 
encoding = tiktoken.encoding_for_model("gpt-4o")

In [96]:
len(encoding.encode(prompt))

321

Use the `encode` function. How many tokens does our prompt have?

* 122
* 222
* `322`
* 422

# Bonus: generating the answer (ungraded)

In [98]:
from openai import OpenAI

In [104]:
client = OpenAI()

In [105]:
response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": prompt}]
    )

In [107]:
response.choices[0].message.content

'You can execute a command in a running docker container by using the `docker exec` command. If the container is already running, you can find the container ID using `docker ps` and then use `docker exec -it <container-id> bash` to execute a command in the specific container.'

# Bonus: calculating the costs (ungraded)

### Current prices for `gpt-3.5-turbo-0125`
* input `$0.50` / `1M` tokens
* output `$1.50` / `1M` tokens

### Suppose that on average per request we send 150 tokens and receive back 250 tokens.

How much will it cost to run 1000 requests?

#### input:

In [109]:
(150 * 1000) * 0.5 / 1000000

0.075

#### output:

In [111]:
(250 * 1000) * 1.5 / 1000000

0.375

#### total for 1000 requests:

In [112]:
(150 * 1000) * 0.5 / 1000000 + (250 * 1000) * 1.5 / 1000000

0.45